# Subthemes OTH, EWC, EXEC

In [42]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from gensim.models.fasttext import FastText

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, codecs

In [2]:
#Import documents
X_train = pd.read_excel('../data/interim/question1_models/X_train.xlsx')
X_valid = pd.read_excel('../data/interim/question1_models/X_valid.xlsx')

y_train = pd.read_excel('../data/interim/question1_models/y_train.xlsx')
y_valid = pd.read_excel('../data/interim/question1_models/y_valid.xlsx')

In [3]:
# from preprocess import Preprocessing
import sys
sys.path.append('../src/data/')
from subset_subtheme_data import subset_data

In [8]:
label_name = 'OTH'
subset_data(label_name, X_train, y_train, X_valid, y_valid)

In [9]:
label_name = 'EWC'
subset_data(label_name, X_train, y_train, X_valid, y_valid)

In [12]:
label_name = 'Exec'
subset_data(label_name, X_train, y_train, X_valid, y_valid)

### Fasttext Embeddings

In [39]:
#load embeddings
embeddings_matrix = {}
f = codecs.open('../data/interim/subthemes/OTH/crawl-300d-2M.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_matrix[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_matrix))

1999996it [03:50, 8669.98it/s] 


found 1999996 word vectors


In [40]:
# create a weight matrix for words in training docs
embedding_matrix_fastext = np.zeros((vocab_size, 300))
for word, i in vect.word_index.items():
    embedding_vector = embeddings_matrix.get(word)
    if embedding_vector is not None:
        embedding_matrix_fastext[i] = embedding_vector

In [52]:
embedding_matrix_fastext

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.0427    , -0.0728    ,  0.0133    , ..., -0.1613    ,
        -0.0381    , -0.0019    ],
       [-0.0538    , -0.1575    , -0.037     , ..., -0.63129997,
         0.14399999,  0.0629    ],
       ...,
       [ 0.0651    ,  0.4427    , -0.25040001, ...,  0.2915    ,
         0.1004    ,  0.34150001],
       [ 0.25709999, -0.039     ,  0.0038    , ..., -0.0509    ,
         0.27509999,  0.1655    ],
       [ 0.42910001,  0.259     , -0.2712    , ...,  0.25600001,
         0.22390001, -0.0841    ]])

In [48]:
embedding_matrix_fastext.shape

(3303, 300)

### OTH

In [15]:
#load data

OTH_X_train = pd.read_excel('../data/interim/subthemes/OTH/X_train_subset.xlsx')
OTH_X_valid = pd.read_excel('../data/interim/subthemes/OTH/X_valid_subset.xlsx')

OTH_Y_train = pd.read_excel('../data/interim/subthemes/OTH/Y_train_subset.xlsx')
OTH_Y_valid = pd.read_excel('../data/interim/subthemes/OTH/Y_valid_subset.xlsx')


In [21]:
OTH_Y_train

,OTH_Other_related,OTH_Positive_comments,OTH_Survey_feedback,OTH_Covid
0,0,1,0,0
1,1,0,0,0
2,0,0,1,0
3,0,0,1,0
4,0,1,0,0
...,...,...,...,...
834,0,1,0,0
835,1,0,0,0
836,0,1,0,0
837,1,0,0,0


**Import preprocessing**

In [16]:
# from preprocess import Preprocessing
from preprocess import Preprocessing

In [20]:
OTH_X_train['preprocessed_comments'] = Preprocessing().general(OTH_X_train['Comment'])


In [22]:
X_train = OTH_X_train[['preprocessed_comments']]
Y_train = np.asarray(OTH_Y_train)

In [24]:
def max_len(x):
    a=x.split()
    return len(a)

max_len = max(OTH_X_train['Comment'].apply(max_len))
max_len

146

In [26]:
vect=Tokenizer()
vect.fit_on_texts(X_train['preprocessed_comments'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

3303


**Padded docs for Oth**

In [43]:
# creating padded dataset for x_train
encoded_docs_train = vect.texts_to_sequences(X_train['preprocessed_comments'])
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_len, padding='post')
print(padded_docs_train)

[[  72 1270 1750 ...    0    0    0]
 [  60   40   46 ...    0    0    0]
 [ 831   36  574 ...    0    0    0]
 ...
 [ 211    1   16 ...    0    0    0]
 [ 594 3297   90 ...    0    0    0]
 [  45   39   40 ...    0    0    0]]


In [45]:
#padded for x_valid
OTH_X_valid['preprocessed_comments'] = Preprocessing().general(OTH_X_valid['Comment'])

encoded_docs_valid = vect.texts_to_sequences(OTH_X_valid['preprocessed_comments'])
padded_docs_valid = pad_sequences(encoded_docs_valid, maxlen=max_len, padding='post')
print(padded_docs_valid)

[[  21   33 1348 ...    0    0    0]
 [  25    0    0 ...    0    0    0]
 [ 100  265 1696 ...    0    0    0]
 ...
 [   6   14  679 ...    0    0    0]
 [ 178   89    4 ...    0    0    0]
 [ 306  637    7 ...    0    0    0]]


In [46]:
padded_docs_train.shape, padded_docs_valid.shape

((839, 146), (215, 146))

In [49]:
#Saving fasttext embeddings
np.save('../data/interim/subthemes/OTH/embedding_matrix_fastext', embedding_matrix_fastext)


#Saving Oth padded
np.save('../data/interim/subthemes/OTH/padded_docs_train', padded_docs_train)
np.save('../data/interim/subthemes/OTH/padded_docs_valid', padded_docs_valid)

### EWC

In [53]:
#load data

EWC_X_train = pd.read_excel('../data/interim/subthemes/EWC/X_train_subset.xlsx')
EWC_X_valid = pd.read_excel('../data/interim/subthemes/EWC/X_valid_subset.xlsx')

EWC_Y_train = pd.read_excel('../data/interim/subthemes/EWC/Y_train_subset.xlsx')
EWC_Y_valid = pd.read_excel('../data/interim/subthemes/EWC/Y_valid_subset.xlsx')


In [54]:
EWC_Y_train

,EWC_Act_on_engagement,EWC_Address_discrimination,EWC_Improve_morale,EWC_Treat_employees_better,EWC_Value_diversity,EWC - Other
0,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,0,0,1,0,0
3,0,0,1,0,0,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
1841,0,0,0,1,0,0
1842,0,0,1,0,0,0
1843,0,0,0,1,0,0
1844,0,0,0,1,0,0


In [55]:
EWC_X_train['preprocessed_comments'] = Preprocessing().general(EWC_X_train['Comment'])


In [56]:
X_train = EWC_X_train[['preprocessed_comments']]
Y_train = np.asarray(EWC_Y_train)

In [57]:
def max_len(x):
    a=x.split()
    return len(a)

max_len = max(EWC_X_train['Comment'].apply(max_len))
max_len

432

In [58]:
vect=Tokenizer()
vect.fit_on_texts(X_train['preprocessed_comments'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

4879


**Padded docs for Ewc**

In [59]:
# creating padded dataset for x_train
encoded_docs_train = vect.texts_to_sequences(X_train['preprocessed_comments'])
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_len, padding='post')
print(padded_docs_train)

[[ 85  48  57 ...   0   0   0]
 [ 61  31  61 ...   0   0   0]
 [ 51  40  32 ...   0   0   0]
 ...
 [231 812   7 ...   0   0   0]
 [ 31  61  31 ...   0   0   0]
 [ 92 893  93 ...   0   0   0]]


In [60]:
#padded for x_valid
EWC_X_valid['preprocessed_comments'] = Preprocessing().general(EWC_X_valid['Comment'])

encoded_docs_valid = vect.texts_to_sequences(EWC_X_valid['preprocessed_comments'])
padded_docs_valid = pad_sequences(encoded_docs_valid, maxlen=max_len, padding='post')
print(padded_docs_valid)

[[  4 475 773 ...   0   0   0]
 [ 61  31  61 ...   0   0   0]
 [103  74   2 ...   0   0   0]
 ...
 [218 967  50 ...   0   0   0]
 [ 17 130  21 ...   0   0   0]
 [133   6  85 ...   0   0   0]]


In [61]:
padded_docs_train.shape, padded_docs_valid.shape

((1846, 432), (436, 432))

In [62]:

#Saving EWC padded
np.save('../data/interim/subthemes/EWC/padded_docs_train', padded_docs_train)
np.save('../data/interim/subthemes/EWC/padded_docs_valid', padded_docs_valid)

### EXEC

In [63]:
#load data

EXEC_X_train = pd.read_excel('../data/interim/subthemes/EXEC/X_train_subset.xlsx')
EXEC_X_valid = pd.read_excel('../data/interim/subthemes/EXEC/X_valid_subset.xlsx')

EXEC_Y_train = pd.read_excel('../data/interim/subthemes/EXEC/Y_train_subset.xlsx')
EXEC_Y_valid = pd.read_excel('../data/interim/subthemes/EXEC/Y_valid_subset.xlsx')


In [64]:
EXEC_Y_train

,Exec_Improve_communication,Exec_Improve_stability,Exec_Strengthen_quality_of_executive_leaders,Exec_other
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
2506,0,0,1,0
2507,1,0,0,0
2508,1,0,0,0
2509,1,0,0,0


In [65]:
EXEC_X_train['preprocessed_comments'] = Preprocessing().general(EXEC_X_train['Comment'])


In [66]:
X_train = EXEC_X_train[['preprocessed_comments']]
Y_train = np.asarray(EXEC_Y_train)

In [67]:
def max_len(x):
    a=x.split()
    return len(a)

max_len = max(EXEC_X_train['Comment'].apply(max_len))
max_len

432

In [68]:
vect=Tokenizer()
vect.fit_on_texts(X_train['preprocessed_comments'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

5073


**Padded docs for EXEC**

In [69]:
# creating padded dataset for x_train
encoded_docs_train = vect.texts_to_sequences(X_train['preprocessed_comments'])
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_len, padding='post')
print(padded_docs_train)

[[172 414 893 ...   0   0   0]
 [ 12  43  13 ...   0   0   0]
 [ 73  13  34 ...   0   0   0]
 ...
 [  5   0   0 ...   0   0   0]
 [  9  85  15 ...   0   0   0]
 [ 29 976 180 ...   0   0   0]]


In [71]:
#padded for x_valid
EXEC_X_valid['preprocessed_comments'] = Preprocessing().general(EXEC_X_valid['Comment'])

encoded_docs_valid = vect.texts_to_sequences(EXEC_X_valid['preprocessed_comments'])
padded_docs_valid = pad_sequences(encoded_docs_valid, maxlen=max_len, padding='post')
print(padded_docs_valid)

[[   4    7   42 ...    0    0    0]
 [ 132   22 1070 ...    0    0    0]
 [ 126  300    2 ...    0    0    0]
 ...
 [  31    1  340 ...    0    0    0]
 [  97   69  247 ...    0    0    0]
 [ 304  214  406 ...    0    0    0]]


In [72]:
padded_docs_train.shape, padded_docs_valid.shape

((2511, 432), (629, 432))

In [73]:

#Saving EWC padded
np.save('../data/interim/subthemes/EXEC/padded_docs_train', padded_docs_train)
np.save('../data/interim/subthemes/EXEC/padded_docs_valid', padded_docs_valid)